In [ ]:
import sys
sys.path.insert(0, sys.path[0].split("notebooks")[0])
import os
import pickle
root = sys.path[0]
furl = os.path.join(root,"build","tdm23_env.vr")
version_min = (0, 0, 7)
with open(furl, "rb") as f:
    tdmvr = pickle.load(f)
if tdmvr.version_info >= version_min:
    print (tdmvr.version_info)
else:
    raise Exception("Please use the right version : tdm23_env.vr-v" + ".".join(map(str, version_min)))
import pandas as pd
from emat.scope.scope import Scope
from emat.analysis.feature_scoring import feature_scores
import emat
notebook = tdmvr.notebook
nb = notebook()


# Load metrics from EMAT database

# TODO: 

## Adding Scenarios: 

- edit db file and scope file location
- edit variable rows
- edit extent of variables

In [2]:
scope_name = 'sens'
exp_db_file =  os.path.join(root,"inputs","tdm23.1.0_sens_20241031.db") 
analysis_scope_file = os.path.join(root,"inputs","tdm23.1.0_sens_20241031.yaml")  
design_name = "uni"

In [ ]:
db = emat.SQLiteDB(exp_db_file, initialize=False)
design_name = db.read_design_names(scope_name=scope_name)[0]
df = db.read_experiment_all(scope_name=scope_name, design_name=design_name, ensure_dtypes=True, source=1)
scope = Scope(scope_file=analysis_scope_file)

In [ ]:
df

In [ ]:
grp_metrics = df.columns[:10].to_list() + scope.get_measure_names()
df = df[grp_metrics]
dff = feature_scores(scope, df, return_type='styled')

In [ ]:
df.DryRun.iloc[0] =  "base"
df.DryRun.iloc[1] = "EB"
df.DryRun.iloc[2:4] = "TNC"
df.DryRun.iloc[4] = "HRT"
df.DryRun.iloc[5] = "WFH"
df.DryRun.iloc[6:8] = "AV"
df.DryRun.iloc[8:10] = "Toll"
df.DryRun.iloc[10:12] = "Fare"
df.DryRun.iloc[12] = "BusLane"
df.DryRun.iloc[13] = "Net"
df.dropna(inplace=True)
df

In [7]:
headers = list(dff.columns) + list(dff.index) + ['DryRun']
abbrevs = headers
schema = []
for i,item in enumerate(headers[:-1]):
    col = {}
    col["index"] = i+1
    col["text"]  = item 
    col["name"]  = abbrevs[i] 
    schema.append(col)
ddf = df[headers]
dict_ixv = ddf.T.to_dict(orient = "list" )
dtsrc = list(dict_ixv.values())
cellheight = 28
echeight = len(ddf.T) * 28 + 500


# Univariate Experiment Analysis

In [ ]:
kwargs = {
    "schema"  : schema,
    "rawData" : dtsrc,
    "CAT_DIM_COUNT" :  len(headers) - 2,
    "CAT_BOOL" :  0,
    "colorhex" :  ['#ce5c5c','#46075a','#46075a','#306a8e','#3abc12','#eed959' ,"#EFE5E5"],
    "tablestr" : ""
}
css = [f"width:800px; height:{echeight}px;",
       "margin-top:323px; margin-right: 100px; width:600px; display:none"]
nb.parallel_bycat(kwargs,css=css)

#